# Symmetrical choiceset:

We want to symmetrically sample both sides of the subjective equality point. 
That means having equal amount of trials in each side of the SV difference. 

This version of the code doesn't use python external libraries. The reason is to optimize the code to make it compatible with online based tasks that use Js. 

JS script data types we are going to use:
- Arrays: written with square brackets. Items are separated by commas. This is analogous to python lists. 

    ``` const cars = ["Saab", "Volvo", "BMW"]; ```
- Objects: JavaScript objects are written with curly braces {}.

Object properties are written as name:value pairs, separated by commas.

This is analogous to python dictionaries. I intend to use them to create the desired choicet. 

        const person = {firstName:"John", lastName:"Doe",  age:50, eyeColor:"blue"}

In [435]:
import csv #This is the only library I will use. However, it is built in python. 
post_mean_gain = [0.45456, 1.0343]
post_mean_loss = [0.896596, -0.765]


The idea is to create an output CSV without using pandas :(
    We are going to do this by using lists and dicitonaries. The idea is to creaate the trials as lists of dictionries

Trials = 
```  trials = 
[{'p_reward': 0.13,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0,
  'SV_reward': 0.7724887228000991,
  'SV_New_Safe': 0.38624436140004953,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 0.12390278661966936},
 {'p_reward': 0.25,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0,
  'SV_reward': 1.4855552361540365,
  'SV_New_Safe': 0.7427776180770183,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 0.5206065839498941},
 {'p_reward': 0.38,
  'value_reward': 50.0,
  'category': 'Extr_pos',
  'ambiguity': 0.0,
  'SV_reward': 2.2580439589541355,
  'SV_New_Safe': 1.1290219794770677,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 1.3052453010492082},
 {'p_reward': 0.5,
...
  'ambiguity': 0.0,
  'SV_reward': 4.4566657084621095,
  'SV_New_Safe': 2.2283328542310548,
  'value_lott_SE': 10.91817090852721,
  'value_safe': 5.806103319838627}]

With that, we can create the CSV at the end

Trials = 
{'category': ['Zero_Reward',  'Zero_Reward',..., 'Int_neg', 'Int_neg','Int_neg'],
 'p_reward': [0.75, 0.5,0.38, 0.25, 0.13, ..., 0.75 0.5, 0.38,0.25,0.13],
 'value_reward': [-0.0,  -5.71, ..., -22.453 , -34.884],
 'ambiguity': [0.0, 0.0,0.0  0.24, 0.5, 0.74, 0.0, 0.0],
 'SV_reward': [0.0, -5.551115123125783e-17, -1.813, -3.62, -2.222, -4.532],
 'SV_New_Safe': [-2.719, -1.8130],
 'value_lott_SE': [-12.723, -12.723, -12.72,
  'value_safe': [-7.20, -0.226,  -2.995, -2.7-2.51]
 }
```

# 1. Input: ADO parameters

In [436]:
alpha_pos, beta_pos = post_mean_gain[0], post_mean_gain[1] #parameters for gains
alpha_neg, beta_neg = post_mean_loss[0], post_mean_loss[1] #parameters for losses

In [437]:
#Declare the variables
p_options = [0.75, 0.5, 0.38, 0.25, 0.13]
amb_options = [0.24, 0.5, 0.74]
A_null = 0.0 # no ambiguity
Vmax = 50.0 #The maximum amount we can pay to participants
Vmin = -50.0 #The minimum amount we can take from participants
Vsafe_pos = 5.0 # The old safe option in positive trials
Vsafe_neg = -5.0 # The old safe option in negative trials

#First, we create an empty dictionary that we will populate with the trials info. 

empty_trials = [{ #An indivual trial consists of the values with the same index in the different lists
  'category': [],
  'p_reward': [],
  'value_lottery': [],
  'ambiguity': [],
  'SV_lottery':[],
  'value_safe': [],
  'SV_safe': []
}]

In [438]:
##FUNCTIONS

#Create the extreme positive gains and losses
def get_extr_rows(p_options, domain='gain'):
    """
    Inputs:
        - p_options = list of probability values (float)
    Output:
        - dictionaries of lists containing the maximum gain or loss. With empty keys that we will populate later on with SV and Vsafw
    """
    extr = empty_trials.copy()
    category = 'Extr_pos' if domain == 'gain' else 'Extr_neg'
    V=[Vmax] * len(p_options) #A list of values with the same length as our p_options
    if domain == 'loss':
            V = [Vmin] * len(p_options)
    extr = {
            'category': [category]*len(p_options),
            'p_reward': [p for p in p_options] ,
            'value_lottery': V,
            'ambiguity': [0.0]*len(p_options),
            
        }
    return extr

#Add ambiguity
def add_ambig(trials,prob_amb=0.50):
    """
    Inputs:
        - trials dictionary
        - amb_options = list of possible ambiguity values
    Output:
        - initial trials dictionary with new ambiguity trials
    """
    V = [trials['value_lottery'][0]]*len(amb_options)
    amb_trials = {
            'p_reward': [prob_amb]*len(amb_options),
            'value_lottery': V,
            'ambiguity': [a for a in amb_options],
        }
    final_dict = {k: trials[k] + amb_trials[k] for k in set(trials) & set(amb_trials)}
    final_dict['category'] = [trials['category'][0]]*len(final_dict['value_lottery'][:])
    return final_dict

#Add Subjective value given participant's alpha and beta
def add_SV(trials, alpha, beta, domain='gain'):
    """
    Inputs:
        - trials dictionary
        - Model parameters
    Output:
        - trials dictionary with subjective value 
    """
    Amp = 1.0
    if domain == 'loss':
          Amp = -1.0
    trials['SV_lott'] = [(p -(beta*A/2)) * (Amp*abs(v) ** alpha) for p, v, A 
                         in zip(trials['p_reward'], trials['value_lottery'], trials['ambiguity'])] #This ZIP method is going to be a PIA in JS. However, I think we can emulate it in JS from scratch 
    return trials  

# Add Vsafe and its subjective value     
def add_Vsafe(trials, alpha,beta, domain = 'gain'):
    """
     This function creates new values of safe options whose subjective value is half of that 
     of the maximum gain or loss SVnew_safe = 0.5*SV_max for each probability. That SV is going to be the SV of subjective equality 

     Inputs:
        - trials dictionary
        - Model parameters
    Output:
        - New safe options
        - SV of SE trials
    """ 
    Amp = 1.0 
    if domain == 'loss':
        Amp = -1.0
    trials['SV_new_safe'] = [sv/2 for sv in trials['SV_lott']]
    trials['value_safe'] = [Amp*(abs(sv))**(1/alpha) for sv in trials['SV_new_safe']]
    return trials


In [442]:
##GAINS
rows_extr_pos = get_extr_rows(p_options)
rows_extr_pos = add_ambig(rows_extr_pos)
rows_extr_pos = add_SV(rows_extr_pos, alpha_pos,beta_pos) 
rows_extr_pos = add_Vsafe(rows_extr_pos, alpha_pos,beta_pos)
print(rows_extr_pos)

{'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74], 'value_lottery': [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], 'category': ['Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos'], 'SV_lott': [4.439612531232422, 2.9597416874882816, 2.249403682491094, 1.4798708437441408, 0.7695328387469532, 2.2250390889196905, 1.4291112738037168, 0.6944086752351256], 'SV_new_safe': [2.219806265616211, 1.4798708437441408, 1.124701841245547, 0.7399354218720704, 0.3847664193734766, 1.1125195444598452, 0.7145556369018584, 0.3472043376175628], 'value_safe': [5.779214056024037, 2.3685382742811, 1.2950266345451085, 0.5155082363421659, 0.12231027909596252, 1.2643682118988953, 0.47740788166001397, 0.09757075252551307]}


In [440]:
##LOSSES
rows_extr_neg = get_extr_rows(p_options, domain = 'loss')
rows_extr_neg = add_ambig(rows_extr_neg) 
rows_extr_neg = add_SV(rows_extr_neg, alpha_neg,beta_neg, domain = 'loss')
rows_extr_neg = add_Vsafe(rows_extr_neg, alpha_neg,beta_neg)

rows_extr_neg


{'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5],
 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74],
 'value_lottery': [-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0],
 'category': ['Extr_neg',
  'Extr_neg',
  'Extr_neg',
  'Extr_neg',
  'Extr_neg',
  'Extr_neg',
  'Extr_neg',
  'Extr_neg'],
 'SV_lott': [-25.023668228005914,
  -16.682445485337276,
  -12.67865856885633,
  -8.341222742668638,
  -4.337435826187692,
  -19.7453424764452,
  -23.063480883478785,
  -26.12637787458671],
 'SV_new_safe': [-12.511834114002957,
  -8.341222742668638,
  -6.339329284428165,
  -4.170611371334319,
  -2.168717913093846,
  -9.8726712382226,
  -11.531740441739393,
  -13.063188937293354],
 'value_safe': [16.74458915384944,
  10.653068363412174,
  7.844090457454431,
  4.917304119496426,
  2.371249269370067,
  12.856488453231144,
  15.288423314523783,
  17.569631173420586]}

# Output CSV

In [441]:
# Extract column names from the keys of the first dictionary
#fieldnames = crdm_trials[0].keys()

# Define the name of the CSV file
#csv_filename = 'crdm_trials.csv'

# Write the data to the CSV file
#with open(csv_filename, mode='w', newline='') as csv_file:
 #   writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header row
  #  writer.writeheader()
    
    # Write the data rows
   # for row in crdm_trials:
    #    writer.writerow(row)

# print(f"CSV file '{csv_filename}' has been created.")